In [83]:
spine_submesh.body_count

1

In [58]:
import numpy as np
import datajoint as dj
import trimesh
from collections import Counter

In [2]:
pinky = dj.create_virtual_module("pinky","microns_pinky")

Connecting celiib@10.28.0.34:3306


In [ ]:
"""
Find where the meshes indices are stored (but these are the indices of the components)
"""
pinky.ProofreadLabelSpineWithInterestSlices() #has both the starting index and the n_triangles

In [ ]:
#DON'T EVEN NEED THIS FUNCTION ANYMORE

def random_submesh(verts,faces,threshold_length = 30000,starting_index=-1,submes_indices_flag=False):
    """
    Purpose: will return the vertices and faces of a submesh of face size threshold_length that was generated from the 
    original mesh passed to it by using a pre-passed or generated index and then building out a connected mesh
    from that face index
    
    Parameters: 
    verts (np.array): vertices of the original mesh
    faces (np.array): faces of the original mesh
    threshold_length (int) : number of faces to be in the submesh (default = 4000)
    starting_index (int) : the face index where the submesh generation will start, if not specified then will just
                            generate the random index
    submes_indices_flag (bool) : a flag to set if you want the submesh indices returned
                            
    Returns: 
    verts_submesh (np.array): vertices of the submesh mesh 
    faces_submesh (np.array): faces of the submesh mesh 
    starting_index (int) : face index where the submesh generation started
    -- optional --
    submesh_faces (np.array): face indexes that were used for creation of submesh 
  
    """
    
    
    def face_lookup(faces_length, face_adjacencies):
        """
        Will create a lookup dictionary that will return the neighbor indexes for each face index
        """
        #create dummy dictionary
        face_adjacency_lookup = dict([(k,[]) for k in range(0,faces_length)])


        #look through the face adjacency and create the lookup table
        for g,h in face_adjacencies:
            face_adjacency_lookup[g].append(h)
            face_adjacency_lookup[h].append(g)

        return face_adjacency_lookup
    
    
    
    """
    Step 1: Generate random starting index if not already
    """
    if starting_index == -1:
        starting_index = random.randint(0,len(faces))
    else:
        #check that the starting index is valid for this mesh
        if starting_index >= len(faces) or starting_index<0:
            raise Exception("Starting Index is invalid for this mesh size ")
    
    print("starting_index = " + str(starting_index))
    
    
    
    
    """
    Step 2: Initialize the arrays to store all submesh face indices and create trimesh object

    """
    outer_ring = [starting_index]
    submesh_faces = {starting_index}
    checked_faces = set()
    new_mesh = trimesh.Trimesh(verts,faces)
    
    #check that the length threshold is not greater than the number of faces, and if it is --> then just change it to size of faces
    if threshold_length > len(faces):
        print(f"Threshold length ({threshold_length}) is greater than number of faces ({len(faces)}) --> so changed threshold_length to ({len(faces)})")
        threshold_length = len(faces)
    
    
    """
    Step 3: Crawl across graph to get the submesh
    3) Build out with the touching faces until you reach your size threshold
    a. Get all of the neighbors of the faces in outer_ring
    b. For each face: 
        - Add all of the adjacent faces to the total faces (if they are not already in there), can just add to set)
        - keep track of the new ones added
        - check that size has not been exceeded, if it has, delete some of the new ones added so that it is under the size

    """
    



    #get the adjacency_matrix
    start_time = time.time()
    face_adjacency_lookup = face_lookup(len(new_mesh.faces),new_mesh.face_adjacency)
    print(f"Total time for lookup table = {time.time() - start_time}")

    start_time = time.time()
    #start loop that finds all of the neighbors for each in the outer-ring

    loop_counter = 0
    while len(submesh_faces) < threshold_length:
        for f in outer_ring:
            #get the neighbors
            current_neighbors = face_adjacency_lookup[f]
            #add the current neighbors to total indices
            submesh_faces.update(current_neighbors)
            #check to see if number of faces is too long
            if len(submesh_faces) > threshold_length:
                print("********** exceeded maximum threshold so now adjusting *************")
                #get the original list before the lists were combined
                old_submesh_faces = submesh_faces.difference(set(current_neighbors))
                #how many spots are available to add:
                spots_available = threshold_length - len(old_submesh_faces)
                #add those amount of faces to the submesh faces
                old_submesh_faces.update(current_neighbors[:spots_available])
                submesh_faces = old_submesh_faces
                break

        #add the outer_ring faces that were processed to the checked list
        #print("outer_ring = " + str(outer_ring))
        checked_faces.update(outer_ring)

        #get the new outer-ring for the next round
        outer_ring = list(submesh_faces.difference(checked_faces))

        #print(f"After loop {loop_counter}, len(submesh_faces) = {len(submesh_faces)}" )
        loop_counter += 1

    print(f"Total time for expansion loop = {time.time() - start_time}")

    #check that the number of submesh faces matches the desired threshold
    if len(submesh_faces) != threshold_length:
        raise Exception(f"Submesh_faces len = {len(submesh_faces)} but threshold_length = {threshold_length}")
        
    rand_submesh = new_mesh.submesh([list(submesh_faces)],append=True)
    
    if submes_indices_flag == True:
        return rand_submesh.vertices,rand_submesh.faces,starting_index,np.array(list(submesh_faces))
    else:
        return rand_submesh.vertices,rand_submesh.faces,starting_index
    

    
    
    

In [29]:
search_key = dict(author_proofreader="js48",status="complete")
complete_table = pinky.ProofreadLabelSpineWithInterestSlices() & search_key
search_dict = dict(search_key,segmentation=3,segment_id=648518346341371119,compartment_type="Oblique",component_index=0)
complete_table

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,author_original name of last editor,author_proofreader name of last editor,starting_index starting index for the slice generation algorithm,n_triangles number of triangles used for slice,date_time the last time it was edited,n_vertices number of vertices as a result of the slice size of n_triangles,vertices label data for the vertices,triangles label data for the faces,triangle_indices the indicies of the triangles used in the slice of the dendrite,edges label data for the edges,status the index of the status descriptor that can be references by the StatusKey,interest_description short description about why this dendrite is interesting
3,648518346341371119,0.35,Oblique,0,computer_Auto,js48,17292,20142,2019-08-22 13:13:55,10066,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349471562,0.35,Apical,0,computer_Auto,js48,108090,30000,2019-08-12 17:19:47,15042,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349471910,0.35,Basal,2,computer_Auto,js48,31952,30000,2019-08-16 12:46:29,15041,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473044,0.35,Apical,0,computer_Auto,js48,7895,30000,2019-08-12 18:41:01,15023,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473160,0.35,Basal,1,computer_Auto,js48,119413,30000,2019-08-12 17:22:11,14999,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473804,0.35,Apical,0,computer_Auto,js48,78047,30000,2019-08-27 16:33:10,15140,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,
3,648518346349473813,0.35,Basal,2,computer_Auto,js48,102954,30000,2019-08-12 17:23:35,15111,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473821,0.35,Apical,0,computer_Auto,js48,144601,30000,2019-08-12 17:25:45,15023,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473835,0.35,Apical,0,computer_Auto,js48,18050,30000,2019-08-12 17:26:29,15021,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473838,0.35,Basal,3,computer_Auto,js48,11184,20963,2019-08-14 12:33:13,10497,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done


In [30]:
"""
way to get the entire mesh:
1) get the id, compartment type and component index, starting index, number of faces, and triangle indices
from the validation proofing table
2) Get the whole mesh from: 
a. pinky.PymeshfixDecimatedExcitatoryStitchedMesh
b. pinky.Decimation35OrphanStitched

3) Use the triangle indices to restrict to the actual portion of submesh

"""

#Example
search_dict = dict(search_key,segmentation=3,segment_id=648518346341371119,compartment_type="Oblique",component_index=0)
verts,faces = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh() & search_dict).fetch1("vertices","triangles")

#get compartment verts and faces
verts_comp,faces_comp = (pinky.CompartmentFinal.ComponentFinal & search_dict).fetch1("vertex_indices","triangle_indices")

#build the trimesh object
new_mesh = trimesh.Trimesh(verts,faces)
new_mesh.show()
compartment_mesh = new_mesh.submesh([faces_comp],append=True)

#now get the submesh according to the triangle indices
tri_indices = (pinky.ProofreadLabelSpineWithInterestSlices & search_dict).fetch1("triangle_indices")
random_mesh = compartment_mesh.submesh([tri_indices],append=True)
random_mesh.show()

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [31]:
# how to get the automatic labels: 
verts_labels,triangles_labels = (pinky.ComponentLabelFinal & search_dict).fetch1("labeled_vertices","labeled_triangles")
        
verts_labels = verts_labels.astype("int")
triangles_labels = triangles_labels.astype("int")


#get the new triangle lables based on the submesh indexes
triangles_labels = triangles_labels[tri_indices]
triangles_labels[triangles_labels == 0] = 17
triangles_labels[triangles_labels == 12] = 14
triangles_labels[triangles_labels == 13] = 14
triangles_labels[triangles_labels == 15] = 14
triangles_labels = triangles_labels
triangles_labels

array([14, 14, 14, ..., 17, 17, 17])

In [32]:
#how to get the proofread labels
tri_labels_proofread = (pinky.ProofreadLabelSpineWithInterestSlices & search_dict).fetch1("triangles")
tri_labels_proofread

array([14, 14, 14, ..., 17, 17, 17], dtype=uint8)

#how to count the overall difference in faces
len(np.where(triangles_labels != tri_labels_proofread))/len(triangles_labels)*100

In [35]:
disagree_mesh = random_mesh.submesh(np.where(triangles_labels != tri_labels_proofread),append=True)
disagree_mesh_disconnected = disagree_mesh.split(only_watertight=False)
len(disagree_mesh_disconnected)

9

In [ ]:
disagree_mesh_groups = trimesh.graph.connected_components(edges=disagree_mesh.face_adjacency,
                                  nodes=np.arange(len(disagree_mesh.faces)),
                                  min_len=1,
                                  engine=None)
list_meshes =  [disagree_mesh.submesh([k],append=True) for k in disagree_mesh_groups]
list_meshes[1].faces.shape


In [ ]:
#now get the shaft only 
shaft_mesh = random_mesh.submesh(np.where(triangles_labels==17),append=True)
shaft_mesh.show()

# Computing the Accuracies

In [ ]:
"""
Total possible labels: 
{12: 'spine_1',
 2: 'shaft',
 18: 'shaft_over_spined',
 5: 'shaft_3',
 0: 'remove_label',
 10: 'error',
 3: 'spine_2',
 4: 'spine_3',
 6: 'spine_4',
 9: 'spine_5',
 11: 'all_one_spine',
 17: 'auto_shaft',
 14: 'auto_spine',
 13: 'spine_head_manual',
 15: 'spine_neck_manual',
 1: 'label_removed'}

"""

In [ ]:
"""
Pseudocode

get the mesh and all of the lists of labels

1) Handle the all-one-spine pieces: get the submesh and split pieces, count the number of pieces as false negatives (SIGNIFICANT SIZED) 
 Eliminate any mesh pieces that are error or no label


For Shaft: 
1) From original mesh extract the indices that were auto-shaft (get the number of faces in submesh to store later)
2) Use the shaft indices to extract the new proofread labels for the autoshaft

--- set size threshold for the spine counting (50)
4) *** DON'T HAVE TO DO ANYMORE **** Get all the all_one_spine label indices --> create submesh from those. Count the number of all_one_spine as a missed spine
5) Get the submesh of anything that is spine_1,spine_2,spine_3,spine_4,spine_5
6) Split the submesh into disjointed parts (using connected_components to get the indices)
7) Loop through the list (with indices)
- if not meet desired threshold then put as 0
- count the number of different spine labels that exist
- store in array
8) Sum the array and add on the all-one-spine number to count the total number missed


For spines: 
1) From original mesh extract indices that were auto-spine (get the number of faces in submesh)
2) Use auto-spine indices to extract the proofread labels for autoshaft
--- set size threshold
4) Dividee the submesh into all disjointed pieces (count the number of pieces and save off)
5) Calculate the number of shaft and over-spine and percentage there are
5) Count the number of pieces that are 70% or greater as false positives and remove those indices from the list (FALSE POSITIVES)
6) For each piece still left in the list
- count the number of different spine labels that exist (if only auto-spine then tally that as perfect)
- store in array
6) Sum up the array and tally up the number that got perfect
"""

In [139]:
# EXAMPLE entry to practice the algorithm on: 
search_key = dict(author_proofreader="js48",status="complete")
complete_table = pinky.ProofreadLabelSpineWithInterestSlices() & search_key
search_dict = dict(search_key,segmentation=3,segment_id=648518346341371119,compartment_type="Oblique",component_index=0)
#search_dict = dict(search_key,segmentation=3,segment_id=648518346349473821,compartment_type="Apical",component_index=0)
search_dict = dict(search_key,segmentation=3,segment_id=648518346349475035,compartment_type="Apical",component_index=2)

significant_size = 50
complete_table

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,author_original name of last editor,author_proofreader name of last editor,starting_index starting index for the slice generation algorithm,n_triangles number of triangles used for slice,date_time the last time it was edited,n_vertices number of vertices as a result of the slice size of n_triangles,vertices label data for the vertices,triangles label data for the faces,triangle_indices the indicies of the triangles used in the slice of the dendrite,edges label data for the edges,status the index of the status descriptor that can be references by the StatusKey,interest_description short description about why this dendrite is interesting
3,648518346341371119,0.35,Oblique,0,computer_Auto,js48,17292,20142,2019-08-22 13:13:55,10066,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349471562,0.35,Apical,0,computer_Auto,js48,108090,30000,2019-08-12 17:19:47,15042,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349471910,0.35,Basal,2,computer_Auto,js48,31952,30000,2019-08-16 12:46:29,15041,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473044,0.35,Apical,0,computer_Auto,js48,7895,30000,2019-08-12 18:41:01,15023,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473160,0.35,Basal,1,computer_Auto,js48,119413,30000,2019-08-12 17:22:11,14999,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473804,0.35,Apical,0,computer_Auto,js48,78047,30000,2019-08-27 16:33:10,15140,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,
3,648518346349473813,0.35,Basal,2,computer_Auto,js48,102954,30000,2019-08-12 17:23:35,15111,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473821,0.35,Apical,0,computer_Auto,js48,144601,30000,2019-08-12 17:25:45,15023,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473835,0.35,Apical,0,computer_Auto,js48,18050,30000,2019-08-12 17:26:29,15021,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done
3,648518346349473838,0.35,Basal,3,computer_Auto,js48,11184,20963,2019-08-14 12:33:13,10497,=BLOB=,=BLOB=,=BLOB=,=BLOB=,complete,done


In [140]:
# get the mesh and the original and proofread labels
"""
way to get the entire mesh:
1) get the id, compartment type and component index, starting index, number of faces, and triangle indices
from the validation proofing table
2) Get the whole mesh from: 
a. pinky.PymeshfixDecimatedExcitatoryStitchedMesh
b. pinky.Decimation35OrphanStitched

3) Use the triangle indices to restrict to the actual portion of submesh

"""

#Example
verts,faces = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh() & search_dict).fetch1("vertices","triangles")

#get compartment verts and faces
verts_comp,faces_comp = (pinky.CompartmentFinal.ComponentFinal & search_dict).fetch1("vertex_indices","triangle_indices")

#build the trimesh object
new_mesh = trimesh.Trimesh(verts,faces)
new_mesh.show()
compartment_mesh = new_mesh.submesh([faces_comp],append=True)

#now get the submesh according to the triangle indices
tri_indices = (pinky.ProofreadLabelSpineWithInterestSlices & search_dict).fetch1("triangle_indices")
random_mesh = compartment_mesh.submesh([tri_indices],append=True)
random_mesh.show()

# how to get the automatic labels: 
verts_labels,triangles_labels = (pinky.ComponentLabelFinal & search_dict).fetch1("labeled_vertices","labeled_triangles")
        
verts_labels = verts_labels.astype("int")
triangles_labels = triangles_labels.astype("int")


#get the new triangle lables based on the submesh indexes
auto_labels = triangles_labels[tri_indices]
auto_labels[auto_labels == 0] = 17
auto_labels[auto_labels == 12] = 14
auto_labels[auto_labels == 13] = 14
auto_labels[auto_labels == 15] = 14
auto_labels = auto_labels
auto_labels

#how to get the proofread labels
tri_labels_proofread = (pinky.ProofreadLabelSpineWithInterestSlices & search_dict).fetch1("triangles")
tri_labels_proofread

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


array([14, 14, 14, ..., 14, 17, 14], dtype=uint8)

In [141]:
#get rid of all of the error and no_label sections: 
labels_dict = dict(
spine_1=12,
    shaft=2,
    shaft_over_spined=18,
    shaft_3=5,
    remove_label=0,
    error=10,
    spine_2=3,
    spine_3=4,
    spine_4=6,
    spine_5=9,
    all_one_spine=11,
    auto_shaft=17,
    auto_spine=14,
    spine_head_manual=13,
    spine_neck_manual=15,
    label_removed=1
)

labels_dict_flipped = dict([(v,k) for k,v in labels_dict.items()])
labels_dict_flipped

{12: 'spine_1',
 2: 'shaft',
 18: 'shaft_over_spined',
 5: 'shaft_3',
 0: 'remove_label',
 10: 'error',
 3: 'spine_2',
 4: 'spine_3',
 6: 'spine_4',
 9: 'spine_5',
 11: 'all_one_spine',
 17: 'auto_shaft',
 14: 'auto_spine',
 13: 'spine_head_manual',
 15: 'spine_neck_manual',
 1: 'label_removed'}

In [142]:
"""
1) Handle the all-one-spine pieces: get the submesh and split pieces, count the number of pieces as false negatives (SIGNIFICANT SIZED) 
 Eliminate any mesh pieces that are error or no label
"""
all_one_spine_n_faces = 0
all_one_spine_n_bodies = 0

x = np.where(tri_labels_proofread == labels_dict["all_one_spine"])[0]
if len(x) > 0:
    all_one_spine_n_faces = len(x)
    #get the submesh
    all_one_spine_mesh = random_mesh.submesh([x],append=True)
    #get the number of splits
    all_one_spine_mesh_split = all_one_spine_mesh.split(only_watertight=False)
    all_one_spine_n_bodies = len(all_one_spine_mesh_split)

print("all_one_spine_n_bodies = " + str(all_one_spine_n_bodies)) 
print("all_one_spine_n_faces = " + str(all_one_spine_n_faces))

print("Done finding all in one spines")


all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines


In [143]:
#np.logical_and(np.logical_and(np.array([True,True,False]),np.array([False,True,False])),np.array([True,False,False]))

In [144]:
Counter(tri_labels_proofread)

Counter({14: 14416, 17: 12025, 10: 1620, 12: 1885, 3: 54})

In [145]:
# """
# Clean the mesh of all the all_one_spine, error and no_label labels
# """

# print(len(tri_labels_proofread))
# clean_indices = np.where(np.logical_and(np.logical_and( 
#                     tri_labels_proofread != labels_dict["error"],tri_labels_proofread != labels_dict["label_removed"] ),
#                     tri_labels_proofread != labels_dict["all_one_spine"]))[0]
# tri_labels_proofread_clean = tri_labels_proofread[clean_indices]
# auto_labels_clean = auto_labels[clean_indices]
# print(len(clean_indices))
# clean_mesh = random_mesh.submesh([clean_indices],append=True)
# len(clean_mesh.faces)
    

In [147]:
"""
For Shaft: 
1) From original mesh extract the indices that were auto-shaft (get the number of faces in submesh to store later)
2) Use the shaft indices to extract the new proofread labels for the autoshaft

--- set size threshold for the spine counting (50)
4) *** DON'T HAVE TO DO ANYMORE **** Get all the all_one_spine label indices --> create submesh from those. Count the number of all_one_spine as a missed spine
5) Get the submesh of anything that is spine_1,spine_2,spine_3,spine_4,spine_5
6) Split the submesh into disjointed parts (using connected_components to get the indices)
7) Loop through the list (with indices)
- if not meet desired threshold then put as 0
- count the number of different spine labels that exist
- store in array
8) Sum the array and add on the all-one-spine number to count the total number missed
"""

#extract the indices that were auto-shaft

auto_shaft_indices = np.where(auto_labels==labels_dict["auto_shaft"])[0]
#random_mesh_auto_shaft_indices = clean_indices[auto_shaft_indices]
n_auto_shaft = len(auto_shaft_indices)
n_auto_shaft

#extract teh proofread shaft labels
tri_labels_proofread_shaft = tri_labels_proofread[auto_shaft_indices]
total_Counter = Counter(tri_labels_proofread_shaft)
shaft_counts = dict([(labels_dict_flipped[k],total_Counter[k]) for k in dict(total_Counter).keys() ])
print("After extracting the shaft after cleaning Distribution is:" + str(shaft_counts) )


#get the indexes of all of the spines
spine_numbers = [labels_dict["spine_" + str(k)] for k in range(1,6)] 
print(spine_numbers)
# np.where(tri_labels_proofread_clean_shaft 
spine_indexes = np.array([])

for iy in spine_numbers: 
    print("Working on spine "+ str(labels_dict_flipped[iy]))
    spine_indexes = np.hstack((spine_indexes, np.where(tri_labels_proofread_shaft==iy)[0]))
    print(f"Len of spine indexes = {len(spine_indexes)}")
spine_indexes = spine_indexes.astype("int")
#check to see if there were any spines: 
total_spine_count_list = []

if (spine_indexes.any()):
    #get the spines only submesh
    spine_submesh = random_mesh.submesh([auto_shaft_indices[spine_indexes]],append=True)
    #divide the spine submesh into disconnected portions
    spine_disconnected_groups = trimesh.graph.connected_components(edges=spine_submesh.face_adjacency,
                                  nodes=np.arange(len(spine_submesh.faces)),
                                  min_len=1,
                                  engine=None)
    #REMINDER: these indices correspond to the spine mesh vertices (which are spine indexes of auto_shaft of random mesh)
    print(len(spine_disconnected_groups))
    
    if len(spine_disconnected_groups)>0:
        for i,spine_group in enumerate(spine_disconnected_groups):
            if len(spine_group) < significant_size:
                print("Size of group too small for a spine")
                continue
            print("Working on shaft spine #" + str(i))
            #iterate through and see what spine labels are present
            original_labels_for_spine = tri_labels_proofread_shaft[spine_indexes[spine_group]]
            print(Counter(original_labels_for_spine))
            
            #see how many spine labels are in the proofread version labels for this spine
            current_spine_count = 0
            for iy in spine_numbers: 
                
                #print("Working on spine "+ str(labels_dict_flipped[iy]))
                if len(np.where(original_labels_for_spine==iy)[0]) > 0: 
                    print(f"Found {iy} in spine group")
                    current_spine_count += 1
                
            total_spine_count_list.append(current_spine_count)
            
print(total_spine_count_list)
total_false_negative = sum(total_spine_count_list)
print("total_false_negative = " + str(total_false_negative))




After extracting the shaft after cleaning Distribution is:{'auto_shaft': 12025, 'spine_1': 989}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 989
Working on spine spine_2
Len of spine indexes = 989
Working on spine spine_3
Len of spine indexes = 989
Working on spine spine_4
Len of spine indexes = 989
Working on spine spine_5
Len of spine indexes = 989
5
Working on shaft spine #0
Counter({12: 180})
Found 12 in spine group
Working on shaft spine #1
Counter({12: 273})
Found 12 in spine group
Working on shaft spine #2
Counter({12: 75})
Found 12 in spine group
Working on shaft spine #3
Counter({12: 305})
Found 12 in spine group
Working on shaft spine #4
Counter({12: 156})
Found 12 in spine group
[1, 1, 1, 1, 1]
total_false_negative = 5


In [148]:
12 in Counter(original_labels_for_spine).keys()

True

In [149]:
"""
For spines: 
1) From original mesh extract indices that were auto-spine (get the number of faces in submesh)
2) Use auto-spine indices to extract the proofread labels for autoshaft
--- set size threshold
4) Dividee the submesh into all disjointed pieces (count the number of pieces and save off)
5) Calculate the number of shaft and over-spine and percentage there are (WILL CALCULATE THIS AS ITERATING THROUGH)
5) Count the number of pieces that are 70% or greater as false positives and remove those indices from the list (FALSE POSITIVES)
6) For each piece still left in the list
- count the number of different spine labels that exist (if only auto-spine then tally that as perfect)
- store in array
6) Sum up the array and tally up the number that got perfect
"""

#extract the indices that were auto-shaft

auto_spine_indices = np.where(auto_labels==labels_dict["auto_spine"])[0]
#random_mesh_auto_shaft_indices = clean_indices[auto_shaft_indices]
n_auto_spine = len(auto_spine_indices)
print("n_auto_spine = " + str(n_auto_spine))

#extract teh proofread spine labels
tri_labels_proofread_spine = tri_labels_proofread[auto_spine_indices]
total_Counter = Counter(tri_labels_proofread_spine)
spine_counts = dict([(labels_dict_flipped[k],total_Counter[k]) for k in dict(total_Counter).keys() ])
print("After extracting the shaft after cleaning Distribution is:" + str(spine_counts) )


#get the indexes of all of the spines
spine_numbers = [labels_dict["spine_" + str(k)] for k in range(1,6)]
spine_numbers_and_spine_overspine = spine_numbers + [labels_dict["auto_spine"],labels_dict["shaft_over_spined"],
                                                     labels_dict["shaft"],
                                                     labels_dict["shaft_3"]
                                                    ]
#print(spine_numbers_and_spine_overspine)
# np.where(tri_labels_proofread_clean_shaft 
spine_indexes = np.array([])

for iy in spine_numbers_and_spine_overspine: 
    #print("Working on spine "+ str(labels_dict_flipped[iy]))
    spine_indexes = np.hstack((spine_indexes, np.where(tri_labels_proofread_spine==iy)[0]))
    print(f"Len of spine indexes = {len(spine_indexes)}")
spine_indexes = spine_indexes.astype("int")
#check to see if there were any spines: 
additional_spine_count_list = []
overspined_list = []
overspined_list_perc = []
spine_size_list = []
false_positive_count = 0
auto_distinct_spines = 0

#
if (spine_indexes.any()):
    #get the spines only submesh
    spine_submesh = random_mesh.submesh([auto_spine_indices[spine_indexes]],append=True)
    #divide the spine submesh into disconnected portions
    spine_disconnected_groups = trimesh.graph.connected_components(edges=spine_submesh.face_adjacency,
                                  nodes=np.arange(len(spine_submesh.faces)),
                                  min_len=1,
                                  engine=None)
    #REMINDER: these indices correspond to the spine mesh vertices (which are spine indexes of auto_shaft of random mesh)
    #print(len(spine_disconnected_groups))
    

    if len(spine_disconnected_groups)>0:
        auto_distinct_spines = len(spine_disconnected_groups)
        for i,spine_group in enumerate(spine_disconnected_groups):
            
            if len(spine_group) < significant_size:
                print("Size of group too small for a spine")
                continue
            
            
            
            
            #print("Working on shaft spine #" + str(i))
            #iterate through and see what spine labels are present
            original_labels_for_spine = tri_labels_proofread_spine[spine_indexes[spine_group]]
            original_labels_counter = Counter(original_labels_for_spine)
            #print(original_labels_counter)
            
            #get the number of shaft faces
            current_shaft_counter = 0
            if labels_dict["shaft_3"] in original_labels_counter.keys():
                current_shaft_counter += original_labels_counter[labels_dict["shaft_3"]]
            if labels_dict["shaft"] in original_labels_counter.keys():
                current_shaft_counter += original_labels_counter[labels_dict["shaft"]]
            
            #count the amount it was labeled as a shaft
            if current_shaft_counter/len(spine_group) > 0.70:
                print("Found false positive for spine " + str(i))
                false_positive_count += 1
                continue
                
            spine_size_list.append(len(spine_group))
                                        
                                        
            if labels_dict["shaft_over_spined"] in original_labels_counter.keys(): 
                overspined_list.append(original_labels_counter[labels_dict["shaft_over_spined"]])
                overspined_list_perc.append(original_labels_counter[labels_dict["shaft_over_spined"]]/len(spine_group))
            else:
                overspined_list.append(0)
                overspined_list_perc.append(0)
            
            #see how many spine labels are in the proofread version labels for this spine
            current_spine_count = 0
            for iy in spine_numbers: 
                
                #print("Working on spine "+ str(labels_dict_flipped[iy]))
                if len(np.where(original_labels_for_spine==iy)[0]) > 0: 
                    #print(f"Found {iy} in spine group")
                    current_spine_count += 1
                
            additional_spine_count_list.append(current_spine_count)
            
print("additional_spine_count_list = " + str(additional_spine_count_list))
            
print("overspined_list = " + str(overspined_list))
            
print("overspined_list_perc = " + str(overspined_list_perc))
            
print("false_positive_count = " + str(false_positive_count))
            
print("auto_distinct_spines = " + str(auto_distinct_spines))

print("spine_size_list = " + str(spine_size_list))
                                                               




n_auto_spine = 15366
After extracting the shaft after cleaning Distribution is:{'auto_spine': 14416, 'spine_1': 896, 'spine_2': 54}
Len of spine indexes = 896
Len of spine indexes = 950
Len of spine indexes = 950
Len of spine indexes = 950
Len of spine indexes = 950
Len of spine indexes = 15366
Len of spine indexes = 15366
Len of spine indexes = 15366
Len of spine indexes = 15366
Size of group too small for a spine
Size of group too small for a spine
Size of group too small for a spine
additional_spine_count_list = [1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
overspined_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [152]:
#the final stats that wanted
#print("additional_spine_count_list = " + str(additional_spine_count_list))

#stats want to keep from the shaft

#stats we want to keep from the spine
            
total_false_negative = sum(total_spine_count_list)
print("total_false_negative = " + str(total_false_negative))

n_merged_spines = sum(additional_spine_count_list)
print("n_merged_spines = " + str(n_merged_spines))
perc_spines_with_merges = len(np.where(np.array(additional_spine_count_list)>0)[0])/len(additional_spine_count_list)
print("perc_spines_with_merges = " + str(perc_spines_with_merges))

#print("overspined_list = " + str(overspined_list))
overspined_faces = np.where(np.array(overspined_list)>0)[0]
n_spines_with_overspined = len(overspined_faces)
print("n_spines_with_overspined = " + str(n_spines_with_overspined))
    
#print("overspined_list_perc = " + str(overspined_list_perc))
#average percentage of overspined 
average_percentage_overspined = np.mean(np.array(overspined_list_perc)[overspined_faces])
print("average_percentage_overspined = " + str(average_percentage_overspined))
            
print("false_positive_count = " + str(false_positive_count))

print("auto_distinct_spines = " + str(auto_distinct_spines))
avg_spine_size = np.mean(spine_size_list)
print("avg_spine_size = " + str(avg_spine_size))

print('n_auto_spine = ' + str(n_auto_spine))
print('n_auto_shaft = ' + str(n_auto_shaft))
                                                       




total_false_negative = 5
n_merged_spines = 11
perc_spines_with_merges = 0.10989010989010989
n_spines_with_overspined = 0
average_percentage_overspined = nan
false_positive_count = 0
auto_distinct_spines = 94
avg_spine_size = 167.34065934065933
n_auto_spine = 15366
n_auto_shaft = 13014


In [119]:
len(np.where(np.array(overspined_list)>0)[0])

1

In [106]:
type(spine_disconnected_groups)

numpy.ndarray

In [66]:
new_array = np.hstack((np.where(tri_labels_proofread_clean_shaft==3)[0], np.where(tri_labels_proofread_clean_shaft==12)[0]))
new_array

array([ 1707,  1709,  1710,  1711,  1714,  1746,  1748,  1751,  1766,
        1780,  1782,  1811,  1813,  1815,  1836,  1839,  1849,  1851,
        1853,  1885,  1889,  1892,  1915,  1934,  1950,  1961,  1970,
        1991,  1993,  2008,  2076,  2092,  2096,  2100,  2140,  2141,
        2162,  2163,  2165,  2167,  2169,  2172,  2173,  2180,  2184,
        2185,  2186,  2190,  2204,  2205,  2207,  2209,  2251,  2260,
        2262,  2280,  2281,  2282,  2290,  2291,  2292,  2293,  2295,
        2300,  2304,  2310,  2313,  2322,  2324,  2367,  2368,  2404,
        2411,  2420,  2423,  2435,  2440,  2493,  2556,  2582,  2883,
        3001,  3562,  3563,  3566,  3576,  3668,  3685,  3751,  3761,
        3765,  3767,  3776,  3781,  3783,  3789,  3794,  3796,  3800,
        3803,  3810,  3811,  3814,  3815,  3816,  3823,  3824,  3826,
        3832,  3835,  3836,  3843,  3844,  3845,  3847,  3854,  3855,
        3867,  3878,  3891,  3893,  3899,  3900,  3903,  3907,  3908,
        3911,  3912,